In [ ]:
import numpy as np
import pandas as pd
import os
import subprocess

# get the path to the root of the repository
root_path = (
    subprocess.check_output(["git", "rev-parse", "--show-toplevel"])
    .decode("utf-8")
    .strip()
)
# set the working directory to the root of the repository
os.chdir(root_path)
# create ./data/09.format_result/ if it doesn't exist
os.makedirs("data/09.format_result", exist_ok=True)

In [ ]:
df = pd.read_csv("./data/07.DEG/star2counts.tsv", delimiter="\t")
# discard GeneType
# df = df.drop(columns=['GeneType'])
# add a column named "Lin-WT Mean", calculate the mean of Lin-WT_*
df["Lin-WT Mean"] = df.loc[:, df.columns.str.startswith("Lin-WT")].mean(axis=1)
# add a column named "Lin-KO4 Mean", calculate the mean of Lin-KO4_*
df["Lin-KO4 Mean"] = df.loc[:, df.columns.str.startswith("Lin-KO4")].mean(axis=1)
# add a column named "Lin-KO5 Mean", calculate the mean of Lin-KO5_*
df["Lin-KO5 Mean"] = df.loc[:, df.columns.str.startswith("Lin-KO5")].mean(axis=1)
deg_AB = pd.read_csv("data/07.DEG/DEG_DESeq2_AB.tsv", sep="\t")
deg_AB = deg_AB[["GeneID", "log2FoldChange", "pvalue", "padj"]]
deg_AB.columns = [str(col) + "_WT/KO4" for col in deg_AB.columns]
deg_AC = pd.read_csv("data/07.DEG/DEG_DESeq2_AC.tsv", sep="\t")
deg_AC = deg_AC[["GeneID", "log2FoldChange", "pvalue", "padj"]]
deg_AC.columns = [str(col) + "_WT/KO5" for col in deg_AC.columns]
deg_BC = pd.read_csv("data/07.DEG/DEG_DESeq2_BC.tsv", sep="\t")
deg_BC = deg_BC[["GeneID", "log2FoldChange", "pvalue", "padj"]]
deg_BC.columns = [str(col) + "_KO4/KO5" for col in deg_BC.columns]
# join df and deg_AB by GeneID and GeneID_WT/KO4
df = df.merge(deg_AB, left_on="GeneID", right_on="GeneID_WT/KO4", how="left")
df = df.merge(deg_AC, left_on="GeneID", right_on="GeneID_WT/KO5", how="left")
df = df.merge(deg_BC, left_on="GeneID", right_on="GeneID_KO4/KO5", how="left")
# drop GeneID.*, but keep GeneID
df = df.drop(columns=df.columns[df.columns.str.startswith("GeneID_")])
print(df.head())

In [ ]:
from Bio import Entrez
from urllib.error import HTTPError
import pickle

os.environ["http_proxy"] = "http://192.168.206.22:7890"


def get_gene_description(ensembl_id):
    # set the email address for Entrez
    Entrez.email = "babaolanqiu@gmail.com"

    # remove the transcript version number from the ensembl ID
    ensembl_id = ensembl_id.split(".")[0]
    # use Entrez to search for the gene information
    handle = Entrez.esearch(db="gene", term=ensembl_id)
    record = Entrez.read(handle)

    try:
        # extract the gene ID from the search results
        gene_id = record["IdList"][0]

        # use Entrez to retrieve the gene information
        handle = Entrez.efetch(db="gene", id=gene_id, retmode="xml")
        record = Entrez.read(handle)

        # extract the gene full name from the gene information
        gene_description = record[0]["Entrezgene_gene"]["Gene-ref"]["Gene-ref_desc"]
        # return pd dataframe of GeneID and Gene description
        return pd.DataFrame(
            {"GeneID": ensembl_id, "Gene description": gene_description}, index=[0]
        )
    except IndexError:
        print(f"No gene information found for Ensembl ID {ensembl_id}")
        return pd.DataFrame(
            {"GeneID": ensembl_id, "Gene description": "None"}, index=[0]
        )
    except HTTPError:
        print(f"HTTPError for Ensembl ID {ensembl_id}")
        # re-try thistry
        return get_gene_description(ensembl_id)


# apply the function to every 5 items in the "GeneID" column of df, use for loop
# gene_descriptions = pd.DataFrame(columns=["GeneID", "Gene description"],index=range(0))
# if gene_descriptions variable don't exist, create it
try:
    gene_descriptions
except NameError:
    gene_descriptions = pd.DataFrame(
        columns=["GeneID", "Gene description"], index=range(0)
    )
for ensembl_id in df["GeneID"]:
    # if GeneID exist in gene_descriptions, skip
    ensembl_id_2 = ensembl_id.split(".")[0]
    if ensembl_id_2 in gene_descriptions["GeneID"].values:
        continue
    # if GeneID not exist in gene_descriptions, apply get_gene_description, and append to the end of gene_descriptions
    gene_descriptions = pd.concat(
        [gene_descriptions, get_gene_description(ensembl_id)], ignore_index=True
    )
    # print progress
    print(f"Progress: {gene_descriptions.shape[0]}/{df.shape[0]}")
    # make a checkpoint every 100 times
    i = gene_descriptions.shape[0]
    # if i is times of 100
    if i % 100 == 0:
        with open("data/09.format_result.pickle", "wb") as f:
            pickle.dump([df, gene_descriptions], f)
        print(f"Progress saved to data/09.format_result.pickle")
# merge df and gene_descriptions by GeneID

In [ ]:
df["GeneID_2"] = df["GeneID"].str.split(".").str[0]
df = pd.merge(df, gene_descriptions, left_on="GeneID_2", right_on="GeneID")
# remove GeneID_2
df = df.drop(columns=["GeneID_2", "GeneID_y"])
# move gene description to the third column
cols = list(df.columns)
cols = cols[:2] + [cols[-1]] + cols[2:-1]
df = df[cols]
# rename GeneID_x to GeneID
df = df.rename(columns={"GeneID_x": "GeneID"})
df.to_csv("data/09.format_result/09.format_result.tsv", sep="\t", index=False)

In [ ]:
df_cpm = pd.read_csv("./data/07.DEG/star2CPM.tsv", sep="\t")
# discard GeneType
# df_cpm = df_cpm.drop(columns=['GeneType'])
# add a column named "Lin-WT Mean", calculate the mean of Lin-WT_*
df_cpm["Lin-WT Mean"] = df_cpm.loc[:, df_cpm.columns.str.startswith("Lin-WT")].mean(
    axis=1
)
# add a column named "Lin-KO4 Mean", calculate the mean of Lin-KO4_*
df_cpm["Lin-KO4 Mean"] = df_cpm.loc[:, df_cpm.columns.str.startswith("Lin-KO4")].mean(
    axis=1
)
# add a column named "Lin-KO5 Mean", calculate the mean of Lin-KO5_*
df_cpm["Lin-KO5 Mean"] = df_cpm.loc[:, df_cpm.columns.str.startswith("Lin-KO5")].mean(
    axis=1
)
deg_AB = pd.read_csv("data/07.DEG/DEG_DESeq2_AB.tsv", sep="\t")
deg_AB = deg_AB[["GeneID", "log2FoldChange", "pvalue", "padj"]]
deg_AB.columns = [str(col) + "_WT/KO4" for col in deg_AB.columns]
deg_AC = pd.read_csv("data/07.DEG/DEG_DESeq2_AC.tsv", sep="\t")
deg_AC = deg_AC[["GeneID", "log2FoldChange", "pvalue", "padj"]]
deg_AC.columns = [str(col) + "_WT/KO5" for col in deg_AC.columns]
deg_BC = pd.read_csv("data/07.DEG/DEG_DESeq2_BC.tsv", sep="\t")
deg_BC = deg_BC[["GeneID", "log2FoldChange", "pvalue", "padj"]]
deg_BC.columns = [str(col) + "_KO4/KO5" for col in deg_BC.columns]
# join df_cpm and deg_AB by GeneID and GeneID_WT/KO4
df_cpm = df_cpm.merge(deg_AB, left_on="GeneID", right_on="GeneID_WT/KO4", how="left")
df_cpm = df_cpm.merge(deg_AC, left_on="GeneID", right_on="GeneID_WT/KO5", how="left")
df_cpm = df_cpm.merge(deg_BC, left_on="GeneID", right_on="GeneID_KO4/KO5", how="left")
# drop GeneID.*, but keep GeneID
df_cpm = df_cpm.drop(columns=df_cpm.columns[df_cpm.columns.str.startswith("GeneID_")])
print(df_cpm.head())

df_cpm["GeneID_2"] = df_cpm["GeneID"].str.split(".").str[0]
df_cpm = pd.merge(df_cpm, gene_descriptions, left_on="GeneID_2", right_on="GeneID")
# remove GeneID_2
df_cpm = df_cpm.drop(columns=["GeneID_2", "GeneID_y"])
# move gene description to the third column
cols = list(df_cpm.columns)
cols = cols[:2] + [cols[-1]] + cols[2:-1]
df_cpm = df_cpm[cols]
# rename GeneID_x to GeneID
df_cpm = df_cpm.rename(columns={"GeneID_x": "GeneID"})
df_cpm.to_csv("data/09.format_result/09.format_result_CPM.tsv", sep="\t", index=False)

In [ ]:
import pickle

# save all the dataframes to a pickle file
with open("data/09.format_result/09.format_result.pickle", "wb") as f:
    pickle.dump([df, gene_descriptions], f)

In [ ]:
import pickle

# load all the dataframes from a pickle file
with open("data/09.format_result/09.format_result.pickle", "rb") as f:
    df, gene_descriptions = pickle.load(f)